In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
INPUT_NODE = 784     # 输入节点
OUTPUT_NODE = 10     # 输出节点
LAYER1_NODE = 500    # 隐藏层数                        
BATCH_SIZE = 100     # 每次batch打包的样本个数        

# 模型相关的参数
LEARNING_RATE = 0.05      
TRAINING_STEPS = 5000 

In [3]:
def inference(input_tensor):
    # 生成隐藏层的参数。
    weights1 = tf.Variable(tf.truncated_normal([INPUT_NODE, LAYER1_NODE], stddev=0.1))
    biases1 = tf.Variable(tf.constant(0.1, shape=[LAYER1_NODE]))
    # 生成输出层的参数。
    weights2 = tf.Variable(tf.truncated_normal([LAYER1_NODE, OUTPUT_NODE], stddev=0.1))
    biases2 = tf.Variable(tf.constant(0.1, shape=[OUTPUT_NODE]))
    
    layer1 = tf.nn.relu(tf.matmul(input_tensor, weights1) + biases1)
    return tf.matmul(layer1, weights2) + biases2

In [4]:
def define_graph():
    x = tf.placeholder(tf.float32, [None, INPUT_NODE], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE], name='y-input')
    
    # 计算不含滑动平均类的前向传播结果
    y = inference(x)
    
    # 定义训练轮数
    global_step = tf.Variable(0, name='global_step', trainable=False)    
    
    # 计算交叉熵及其平均值
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    
    # 优化损失函数
    train_op = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(
        cross_entropy_mean, global_step=global_step)
    
    # 计算正确率
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    return x, y_, train_op, accuracy

In [5]:
def train(x, y_, train_op, accuracy, mnist):
    # 初始化会话，并开始训练过程。
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    validate_feed = {x: mnist.validation.images, y_: mnist.validation.labels}
            
    # 循环的训练神经网络。
    for i in range(TRAINING_STEPS):
        if i % 1000 == 0:
            validate_acc = sess.run(accuracy, feed_dict=validate_feed)
            print("After %d training step(s), validation accuracy using average model is %g " % (i, validate_acc))
            
        xs, ys = mnist.train.next_batch(BATCH_SIZE)
        sess.run(train_op,feed_dict={x:xs,y_:ys})
    
    return sess

In [6]:
def test(x, y_, accuracy, mnist, sess):
    test_feed = {x: mnist.test.images, y_: mnist.test.labels} 
    test_acc = sess.run(accuracy,feed_dict=test_feed)
    print(("After %d training step(s), test accuracy using average model is %g" %(TRAINING_STEPS, test_acc)))

In [7]:
def main(argv=None):
    mnist = input_data.read_data_sets("MNIST_data", one_hot=True)
    x, y_, train_op, accuracy = define_graph()
    sess = train(x, y_, train_op, accuracy, mnist)
    test(x, y_, accuracy, mnist, sess)
    sess.close()
    
if __name__=='__main__':
    main()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
After 0 training step(s), validation accuracy using average model is 0.0584 
After 1000 training step(s), validation accuracy using average model is 0.936 
After 2000 training step(s), validation accuracy using average model is 0.9518 
After 3000 training step(s), validation accuracy using average model is 0.958 
After 4000 training step(s), validation accuracy using average model is 0.9652 
After 5000 training step(s), test accuracy using average model is 0.9639
